In [42]:
import numpy as np
import os
import pyloudnorm as pyln
from openai import OpenAI
import librosa
import soundfile as SF

import pyloudnorm as pyln

import pyfar as pf
import pandas as pd


import sys, random  
sys.path.insert(0, '../')
import utils
from utils.audio_generation import sample, get_model
from utils.audio_processing import compress_spectrogram_simple, compress_spectrogram_with_centroid, equalize_audio, butter_bandpass_filter, pitch_shift_centroid, equalize_audio

from IPython.display import Audio, display
from ipywidgets import widgets

%matplotlib inline
import matplotlib.pyplot as plt

In [14]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [16]:
#Audio params
loudness_dbfs = -14.0
sample_rate = 16000
stft_channels = 1024
hop_length = 128

# Diffusion params
guidance_scale = 3
n_candidates = 1
batch_size = 1
ddim_steps = 100

In [20]:
loudness_meter = pyln.Meter(sample_rate)

In [41]:
os.makedirs('samples/imu_tones_equalized', exist_ok=True)
for i in range(10,1000,1):
    audio = librosa.tone(i,sr=sample_rate, duration=2)
    audio = equalize_audio(audio, sample_rate, loudness_meter=None, loudness=-25)
    
    sf.write('samples/imu_tones_equalized/'+str(i)+'.wav', audio, samplerate=sample_rate)
# utils.plot_single(audio, '100')

In [44]:
def renormalize(n, range1, range2):
    delta1 = range1[1] - range1[0]
    delta2 = range2[1] - range2[0]
    return (delta2 * (n - range1[0]) / delta1) + range2[0]

In [56]:
df = pd.read_csv('freq-response-DRAKE-MF-BLACK-raw-1721643327.220492.csv')
new_vals = []
for i in df['accVal']:
    new_vals.append(-1*renormalize(i, (np.min(df['accVal']),np.max(df['accVal'])), (0,0.5)))

In [57]:
os.makedirs('samples/imu_tones_equalized_MF_0.5_filterstep10', exist_ok=True)
for i in range(10,500,1):
    audio = librosa.tone(i,sr=sample_rate, duration=2)
    y = pf.classes.audio.Signal(audio, sampling_rate=sample_rate)
    for ind, freq_c in enumerate(df['freq']):
        if ind%10 == 0:
            y = pf.dsp.filter.bell(signal=y, center_frequency=freq_c, gain=new_vals[ind], quality=4)

    
    sf.write('samples/imu_tones_equalized_MF_0.5_filterstep10/'+str(i)+'.wav', y._data[0], samplerate=sample_rate)
# utils.plot_single(audio, '100')




In [59]:
# for ind, freq_c in enumerate(df['freq']):
#     print(freq_c)